In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader 
import timm
from torchvision import transforms
import os
from PIL import Image
import pandas as pd
from training_utils import *
from sklearn.metrics import accuracy_score, roc_auc_score, fbeta_score, precision_score, recall_score, confusion_matrix

/home/giuto/Desktop/Uni/Magistrale/1° Anno/II° Semestre/Computer vision/Murgia-AI-Wildlife-Track/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_model(model, dataloader, device, label_map):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Ricava classi presenti effettivamente nel test
    unique_labels = sorted(list(set(all_labels + all_preds)))
    inv_label_map = {v: k for k, v in label_map.items()}
    class_names = [inv_label_map[i] for i in unique_labels]

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, labels=unique_labels, target_names=class_names))

    print("\nConfusion Matrix:")
    cm = confusion_matrix(all_labels, all_preds, labels=unique_labels)
    print(cm)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
label_map = {
    'badger': 0,
    'bird': 1,
    'boar': 2,
    'butterfly': 3,
    'cat': 4,
    'dog': 5,
    'fox': 6,
    'lizard': 7,
    'podolic_cow': 8,
    'porcupine': 9,
    'weasel': 10,
    'wolf': 11
}

In [5]:
# Dataset and DataLoader

train_df = pd.read_csv("data/augmented_train.csv")
val_df = pd.read_csv("data/augmented_val.csv")
test_df = pd.read_csv("data/augmented_test.csv")

train_ds = AnimalDataset(train_df, "data/labeled_img_aug", transform=transform, label_map=label_map, crop_bbox=False)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

val_ds = AnimalDataset(val_df, "data/labeled_img_aug", transform=transform, label_map=label_map, crop_bbox=False)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

test_ds = AnimalDataset(test_df, "data/labeled_img_aug", transform=transform, label_map=label_map, crop_bbox=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

### Trining of Vit model with frozen backbone

In [6]:
num_classes = len(train_ds.label_map)

model_frozen = timm.create_model('vit_base_patch16_224', pretrained=True)
model_frozen.head = nn.Linear(model_frozen.head.in_features, num_classes)  # sostituisci testa

# Frozen the model except for the head
for param in model_frozen.parameters():
    param.requires_grad = False
# Unfreeze the head
for param in model_frozen.head.parameters():
    param.requires_grad = True

model_frozen = model_frozen.to(device)

optimizer_frozen = torch.optim.Adam(filter(lambda p: p.requires_grad, model_frozen.parameters()), lr=1e-3)
criterion_frozen = nn.CrossEntropyLoss()

In [7]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model_frozen, train_loader, optimizer_frozen, criterion_frozen, device)
    val_loss, val_acc = eval_model(model_frozen, val_loader, criterion_frozen, device)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")

test_loss, test_acc = eval_model(model_frozen, test_loader, criterion_frozen, device)
print(f"\nTest finale → Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")

print("\nMetriche di classificazione dettagliate:")
evaluate_model(model_frozen, test_loader, device, train_ds.label_map)

Epoch 1/5 - Train loss: 1.0574, acc: 0.7143 | Val loss: 0.3357, acc: 0.8896
Epoch 2/5 - Train loss: 0.2121, acc: 0.9570 | Val loss: 0.1957, acc: 0.9221
Epoch 3/5 - Train loss: 0.1180, acc: 0.9750 | Val loss: 0.1319, acc: 0.9675
Epoch 4/5 - Train loss: 0.0775, acc: 0.9861 | Val loss: 0.1099, acc: 0.9545
Epoch 5/5 - Train loss: 0.0555, acc: 0.9945 | Val loss: 0.0984, acc: 0.9610

Test finale → Loss: 0.1190 | Accuracy: 0.9742

Metriche di classificazione dettagliate:

Classification Report:
              precision    recall  f1-score   support

      badger       0.88      1.00      0.93         7
        bird       0.94      1.00      0.97        30
        boar       1.00      0.87      0.93        15
   butterfly       1.00      1.00      1.00         7
         cat       1.00      1.00      1.00         8
         dog       1.00      1.00      1.00         8
         fox       1.00      0.75      0.86         8
      lizard       1.00      1.00      1.00         7
 podolic_cow       1

### Trining of Vit model updating all weight

In [8]:
model_finetune = timm.create_model('vit_base_patch16_224', pretrained=True)
model_finetune.head = nn.Linear(model_finetune.head.in_features, num_classes)

# Unlock all parameters for finetuning
for param in model_finetune.parameters():
    param.requires_grad = True

model_finetune = model_finetune.to(device)

optimizer_finetune = torch.optim.Adam(model_finetune.parameters(), lr=1e-4)
criterion_finetune = nn.CrossEntropyLoss()

In [9]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model_finetune, train_loader, optimizer_finetune, criterion_finetune, device)
    val_loss, val_acc = eval_model(model_finetune, val_loader, criterion_finetune, device)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")

test_loss, test_acc = eval_model(model_finetune, test_loader, criterion_finetune, device)
print(f"\nTest finale → Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")

print("\nMetriche di classificazione dettagliate:")
evaluate_model(model_finetune, test_loader, device, train_ds.label_map)

Epoch 1/5 - Train loss: 0.7923, acc: 0.7462 | Val loss: 0.2673, acc: 0.8896
Epoch 2/5 - Train loss: 0.2636, acc: 0.9085 | Val loss: 0.2644, acc: 0.9026
Epoch 3/5 - Train loss: 0.1083, acc: 0.9653 | Val loss: 0.1082, acc: 0.9675
Epoch 4/5 - Train loss: 0.0568, acc: 0.9861 | Val loss: 0.1329, acc: 0.9675
Epoch 5/5 - Train loss: 0.0155, acc: 0.9972 | Val loss: 0.1797, acc: 0.9545

Test finale → Loss: 0.0919 | Accuracy: 0.9742

Metriche di classificazione dettagliate:

Classification Report:
              precision    recall  f1-score   support

      badger       1.00      0.86      0.92         7
        bird       1.00      1.00      1.00        30
        boar       1.00      0.87      0.93        15
   butterfly       1.00      1.00      1.00         7
         cat       1.00      1.00      1.00         8
         dog       1.00      1.00      1.00         8
         fox       0.89      1.00      0.94         8
      lizard       1.00      1.00      1.00         7
 podolic_cow       1